# Clean up Texas Ownership Data

The Texas ICFIID Directory comes from [Texas Health and Human Services](https://www.hhs.texas.gov/providers/long-term-care-providers/intermediate-care-facilities-icfiid). It is a listing of all currently open facilities in the state. It was downloaded on Sept. 20, 2021. This notebook merges that data, plus additional research done by BuzzFeed News about facilities that were closed prior to that directory's publication, with the QCOR data.

In [1]:
import pandas as pd

## Load State Data

In [2]:
RAW_OWNERSHIP_DIR = "../../data/ownership/raw/"
INTERMEDIATE_OWNERSHIP_DIR = "../../data/ownership/intermediate/"

In [3]:
tex_df_raw = pd.read_excel(
    RAW_OWNERSHIP_DIR + "Texas ICFIID Directory as of 9.20.21.xlsx", 
    sheet_name="ICF-IID Directory", 
    skiprows=1
)

In [4]:
tex_df_raw.head()

,Facility Name,Facility_ID,Program Type,Region,HHSC SubOffice,County_,Service Type,Facility Licensed,Facility Certified,License Number,...,Type of Entity,Owner Mailing1,Owner Mailing City,Owner Mailing STATE,Owner Mailing Zipcode,Owner PHONE,Owner FAX,Administrator,Management Company_,Provider Email
0,ELKHART INN COMMUNITY HOME,3868,ICF/IID,4,ICF/IID TEAM,ANDERSON,PRIVATELY OWNED,YES,YES,146727.0,...,LIMITED PARTNERSHIP,"805 N WHITTINGTON PARKWAY, SUITE 400",LOUISVILLE,KY,40222,(409) 840-9111,(502) 394-2285,KATHY NEWSOME,NaN,licenses@brightspringhealth.com
1,CRESTVIEW COMMUNITY HOME,7294,ICF/IID,4,ICF/IID TEAM,ANDERSON,PRIVATELY OWNED,YES,YES,146582.0,...,LIMITED PARTNERSHIP,"805 N WHITTINGTON PARKWAY, SUITE 400",LOUISVILLE,KY,40222,(409) 840-9111,(502) 394-2285,PATRICIA ELAINE KING,NaN,bbuckman@rescare.com
2,MAVERICK COMMUNITY HOME,3685,ICF/IID,4,ICF/IID TEAM,ANDERSON,PRIVATELY OWNED,YES,YES,146579.0,...,LIMITED PARTNERSHIP,"805 N WHITTINGTON PARKWAY, SUITE 400",LOUISVILLE,KY,40222,(409) 840-9111,(502) 394-2285,PATRICIA ELAINE KING,NaN,bbuckman@rescare.com
3,REDWOOD TERRACE COMMUNITY HOME,7456,ICF/IID,4,ICF/IID TEAM,ANDERSON,PRIVATELY OWNED,YES,YES,146699.0,...,LIMITED PARTNERSHIP,"805 N WHITTINGTON PARKWAY, SUITE 400",LOUISVILLE,KY,40222,(409) 840-9111,(502) 394-2285,PATRICIA ELAINE KING,NaN,NaN
4,WESTWOOD COMMUNITY HOME,3928,ICF/IID,4,ICF/IID TEAM,ANDERSON,PRIVATELY OWNED,YES,YES,146777.0,...,LIMITED PARTNERSHIP,"805 N WHITTINGTON PARKWAY, SUITE 400",LOUISVILLE,KY,40222,(409) 840-9111,(502) 394-2285,STEPHEN KOONCE,NaN,licenses@rescare.com


In [5]:
tex_df_raw.columns

Index(['Facility Name', 'Facility_ID', 'Program Type', 'Region',
       'HHSC SubOffice', 'County_', 'Service  Type', 'Facility  Licensed',
       'Facility Certified', 'License Number', 'License Effective Date',
       'License Expiration Date', 'Physical Address', 'Physical Address CITY',
       'Physical Address State', 'Physical Address Zipcode', 'Geo Location',
       'Facility Phone Number', 'Facility Fax', 'Total Licensed Capacity',
       'Licensed Only Beds', 'Medicaid Only Beds', 'Medicare Only Beds',
       'Medicaid / Medicare Beds', 'Alzheimer Capacity', 'ICFIID Beds',
       'Owner_', 'Type of Entity', 'Owner Mailing1', 'Owner Mailing City',
       'Owner Mailing STATE', 'Owner Mailing Zipcode', 'Owner PHONE',
       'Owner FAX', 'Administrator', 'Management Company_', 'Provider Email'],
      dtype='object')

In [6]:
tex_df_raw["Owner_"].value_counts().head(10)

EDUCARE COMMUNITY LIVING CORPORATION - TEXAS         95
EDUCARE COMMUNITY LIVING LIMITED PARTNERSHIP         75
D&S RESIDENTIAL SERVICES, LP                         46
DAYBREAK, INC.                                       26
EVERGREEN PRESBYTERIAN MINISTRIES OF TEXAS INC       26
EDUCARE COMMUNITY LIVING CORPORATION - GULF COAST    24
KENMAR RESIDENTIAL SERVICES, INC.                    23
ROCK HOUSE SUPPORT SERVICES INC                      22
DADS                                                 17
CALAB, INC.                                          15
Name: Owner_, dtype: int64

In [7]:
len(tex_df_raw)

788

In [8]:
def clean_facility_name(facility_name):
    facility_name = facility_name.strip("?")
    fac_name_replacements = {}
    if facility_name in fac_name_replacements.keys():
        facility_name = fac_name_replacements[facility_name]
    return facility_name

In [9]:
def clean_street_address(address):
    # Here we manually clean address syntax and errors 
    # that we found through research
    address_replacements = {
        "6844 US-69": "HWY 69 N",
        "5310 BLANCO AVE": "5310 BLANCO ST",
        "1502 MOCKINGBIRD LN": "1502 MOCKINGBIRD DRIVE",
        "5124 QUEEN ANN CT": "5124 QUEEN ANN DRIVE",
        "31111 STELLA LN": "31111 STELLA LANE",
        "157 BESS": "157 BESS STREET",
        "17626 KINGS CT": "17626 KINGS COURT",
        "526 HAWTHORNE ST": "3302 BELLPEPPER WAY",
        "707 S TEMPLE AVENUE": "TEMPLE AVE AT DELTA HEIGHTS",
        "2812 CALIFORNIA LN": "2812 CALIFORNIA",
        "2011 W. US HIGHWAY 90": "2011 W. US HIGHWAY 90", # Felice Schwartz Community Residence
        "2021 W. US HIGHWAY 90": "2021 W. US HIGHWAY 90", # Hirsch Schwartz Community Residence
        "6225 S HULEN": "2620 MEADERS",
        "?2812 SHAFFER LN?": "2812 SHAFFER LANE",
        "?3506 GARBER CIR?CLE": "3506 GARBER CIRCLE",
        "?2525 PRESTON? AVENUE": "2525 PRESTON",
        "?3323 GAIL LANE": "3323 GAIL LN",
        "?100 PATTI J? STREET": "100 PATTI J STREET",
        "?3600 NEW COPELAND RD?": "3600 NEW COPELAND RD",
        "1825 CAMPBELL": "9438 CAMPBELL RD", # Campbell House
        "1102 OAKLAWN": "7008 VOLLEY COURT", # Oaklawn House
        "1312 TAMMY ST.": "18937 K-Z ROAD", # Tammy House
        "5835 NW CR 2091": "7336 CHANNEL VIEW DRIVE" # Sunset Acres House
    }
    if address in address_replacements.keys():
        address = address_replacements[address]
    return address

In [10]:
tex_df = (
    tex_df_raw
    .assign(clean_name = lambda df: df["Facility Name"].apply(lambda x: clean_facility_name(x)))
    .assign(street_address = lambda df: df["Physical Address"].apply(lambda x: clean_street_address(x)))
)

Texas facility names are not unique. It's only an issue for BOULDER OAKS and HORIZON HOUSE. This is why the joins are on name and address.

In [11]:
tex_df.groupby(["clean_name"])["Owner_"].nunique().sort_values(ascending=False).head(10)

clean_name
BOULDER OAKS             2
HORIZON HOUSE            2
100 PATTI J STREET       1
NORTHCREST GROUP HOME    1
NOPAL COMMUNITY HOME     1
NORRIS PLACE             1
NORTH HOUSE              1
NORTH ROCK 1             1
NORTH ROCK 2             1
NORTH SHEPHERD HOUSE     1
Name: Owner_, dtype: int64

## Load CMS QCOR Data

In [12]:
tx_facs = (
    pd
    .read_csv("../../data/qcor/facilities.csv", parse_dates=["termination_date"])
    .loc[
        lambda x:
            (x["state"] == "TX") &
            (x["termination_code"].isnull() | (x["termination_date"] >= "2017-03-01"))
    ]
    .assign(street_address = lambda df: df["address"].apply(lambda x: x.split("\n")[0]))
)

In [13]:
len(tx_facs)

829

In [14]:
tx_facs.head()

,name,provider_id,type,region,state,address,phone,particip_date,certified_beds,hospital_based,ownership_type,termination_code,termination_date,street_address
5447,ABILENE STATE SUPPORTED LIVING CENTER,45G001,Intermediate Care Facilities for Individuals w...,(VI) Dallas,TX,"2501 MAPLE ST\nABILENE, TX 79602",325 692-4053,1982-06-15,269,No,Government,NaN,NaT,2501 MAPLE ST
5448,AUSTIN STATE SUPPORTED LIVING CENTER,45G002,Intermediate Care Facilities for Individuals w...,(VI) Dallas,TX,"2203 W 35TH ST\nAUSTIN, TX 78767",512 454-4731,1983-02-01,474,No,Government,NaN,NaT,2203 W 35TH ST
5449,BRENHAM STATE SUPPORTED LIVING CENTER,45G003,Intermediate Care Facilities for Individuals w...,(VI) Dallas,TX,"HIGHWAY 36 SOUTH\nBRENHAM, TX 77833",979 836-4511,1983-06-01,450,No,Non-Profit,NaN,NaT,HIGHWAY 36 SOUTH
5450,DENTON STATE SUPPORTED LIVING CENTER,45G004,Intermediate Care Facilities for Individuals w...,(VI) Dallas,TX,"3980 STATE SCHOOL RD\nDENTON, TX 76210",940 891-0342,1982-10-14,716,No,Government,NaN,NaT,3980 STATE SCHOOL RD
5451,LUFKIN STATE SUPPORTED LIVING CENTER,45G005,Intermediate Care Facilities for Individuals w...,(VI) Dallas,TX,"HWY 69 N\nLUFKIN, TX 75901",936 853-8350,1983-06-18,486,No,Government,NaN,NaT,HWY 69 N


In [15]:
tx_facs_with_owner = (
    pd
    .merge(
        tx_facs,
        tex_df[["clean_name", "Owner_", "street_address"]],
        how="left",
        left_on=["name","street_address"],
        right_on=["clean_name","street_address"]
    )
    .rename(columns={"Owner_": "legal_owner"})
    .assign(
        is_brightspring = lambda df: df["legal_owner"].isin([
            "EDUCARE COMMUNITY LIVING CORPORATION - TEXAS",
            "EDUCARE COMMUNITY LIVING LIMITED PARTNERSHIP",
            "EDUCARE COMMUNITY LIVING CORPORATION - GULF COAST"
        ])
    )
    .set_index("provider_id") # setting this to make the assignment below easier
)

In [16]:
tx_facs_with_owner.sample(5, random_state=0)

,name,type,region,state,address,phone,particip_date,certified_beds,hospital_based,ownership_type,termination_code,termination_date,street_address,clean_name,legal_owner,is_brightspring
provider_id,,,,,,,,,,,,,,,,
45H492,EVERGREEN WAGNER COMMUNITY HOME,Intermediate Care Facilities for Individuals w...,(VI) Dallas,TX,"7905 PEREGRINE TRAIL\nARLINGTON, TX 76001",817 477-5600,2004-05-31,6,No,For Profit,NaN,NaT,7905 PEREGRINE TRAIL,EVERGREEN WAGNER COMMUNITY HOME,EVERGREEN PRESBYTERIAN MINISTRIES OF TEXAS INC,False
45G184,NEW HOPE COMMUNITY LIVING III,Intermediate Care Facilities for Individuals w...,(VI) Dallas,TX,"3204 DUNDEE ST\nEL PASO, TX 79925",915 843-7783,1985-10-03,6,No,For Profit,NaN,NaT,3204 DUNDEE ST,NEW HOPE COMMUNITY LIVING III,LANCE R HUGHES MANAGEMENT TRUST,False
45G010,LUBBOCK STATE SUPPORTED LIVING CENTER,Intermediate Care Facilities for Individuals w...,(VI) Dallas,TX,"3401 N UNIVERSITY AVE\nLUBBOCK, TX 79415",806 763-7041,1982-08-06,436,No,Government,NaN,NaT,3401 N UNIVERSITY AVE,LUBBOCK STATE SUPPORTED LIVING CENTER,DADS,False
45G376,EVERGREEN MEADOW LANE COMMUNITY HOME,Intermediate Care Facilities for Individuals w...,(VI) Dallas,TX,"#20 MEADOW LN\nTEXARKANA, TX 75503",903 792-2529,1990-02-01,6,No,Non-Profit,NaN,NaT,#20 MEADOW LN,EVERGREEN MEADOW LANE COMMUNITY HOME,EVERGREEN PRESBYTERIAN MINISTRIES OF TEXAS INC,False
45H102,103 E VISER ST,Intermediate Care Facilities for Individuals w...,(VI) Dallas,TX,"103 E VISER STREET\nMADISONVILLE, TX 77864",888 775-5135,1993-09-03,6,No,For Profit,NaN,NaT,103 E VISER STREET,103 E VISER ST,"DAYBREAK, INC.",False


## Incorporate Manual Research

BuzzFeed News reporters tracked down information about closed ICFs in Texas. A few of those facilities were run by BrightSpring (or its predecessors) prior to closing. Below we set those values.

In [17]:
# Setting manually the data for the three closed facilities run by BrightSpring predecessors
closed_brightspring_fac_ids = ["45G066", "45G201", "45G231"]
for cbfi in closed_brightspring_fac_ids:
    tx_facs_with_owner.at[cbfi, "is_brightspring"] = True

On Nov. 24, 2021 Texas provided additional information about those closed facilities. It included closure dates for 16 facilities in the QCOR dataset.

In [18]:
closed_facs = pd.read_csv(INTERMEDIATE_OWNERSHIP_DIR + "closed-texas-facilities.csv").loc[
    lambda x: x["status"].str.contains("Closed")
]

In [19]:
len(closed_facs)

16

In [20]:
closed_facs.head()

,provider_id,name,status,closed_date
0,45G166,1801 BRANARD,Closed 08/01/2021,8/1/2021
1,45G368,ALDWYCHE,Closed 11/01/2020,11/1/2020
2,45G048,ALTERNATE LIVING FACILITY I,Closed 02/14/2021,2/14/2021
3,45H096,COUNCIL OAKS AT ALMARION WAY,Closed 03/18/2021,3/18/2021
4,45G883,EVERGREEN STONE TRAIL COMMUNITY HOME,Closed 10/27/2020,10/27/2020


In [21]:
# Setting manual termination dates
for i, r in closed_facs.iterrows():
    tx_facs_with_owner.at[r["provider_id"], "termination_date"] = r["closed_date"]

In [22]:
tex_df.loc[
    lambda x: x["Physical Address"].str.contains("GAIL")
]

,Facility Name,Facility_ID,Program Type,Region,HHSC SubOffice,County_,Service Type,Facility Licensed,Facility Certified,License Number,...,Owner Mailing City,Owner Mailing STATE,Owner Mailing Zipcode,Owner PHONE,Owner FAX,Administrator,Management Company_,Provider Email,clean_name,street_address
568,3323 GAIL LANE,7342,ICF/IID,4,ICF/IID TEAM,SMITH,PRIVATELY OWNED,YES,YES,308125.0,...,LUFKIN,TX,75901,(936) 639-1615,(936) 639-1632,ED E NOXON,NaN,enoxon@communityaccessinc.com,3323 GAIL LANE,3323 GAIL LANE


In [23]:
len(
    tx_facs_with_owner
    .reset_index()
    .loc[
        lambda x: 
        x["legal_owner"].isnull() 
    ]
)

43

In [24]:
# output for analysis
tx_facs_with_owner.reset_index()[[
    "name", "provider_id", "type", "region", "state", "address",
    "phone", "address", "particip_date", "certified_beds",
    "hospital_based", "ownership_type", "termination_code",
    "termination_date", "legal_owner", "is_brightspring"
]].to_csv("../../data/ownership/final/tx.csv", index=None)

---

---

---